Example workflow using pycaret, and Jimmy's "utility_forecast_analysis.ipynb"

Author: Hayden Fiege
Date: May 2 2021

In [23]:
import pandas as pd
import numpy as np
from math import log10
import sys
import seaborn as sns
from datetime import datetime



from functools import reduce
import matplotlib.pyplot as plt
%matplotlib notebook
from weather_scrape import *
from electricity_scrape import * 


Import weather_scrape.py and electricity_scrape.py files containing functions to scrape data

In [2]:
# weather params
city = 'yyc'
startdate = '2000-01-01'
enddate = '2020-12-31'
dateformat = '%Y-%m-%d'
contenttype = 'html'
#get raw data into pd dataframe, timeframe = 2 is daily data
weather_yyc_df  =  weather_dl_range(startdate, enddate, dateformat, city, 2, clean=True)
city = 'yeg'
weather_yeg_df  =  weather_dl_range(startdate, enddate, dateformat, city, 2, clean=True)
city = 'ymm'
weather_ymm_df  =  weather_dl_range(startdate, enddate, dateformat, city, 2, clean=True)

Getting data for YYC, station ID 50430
downloading daily data for year 2000
data for year 2000 downloaded
sleep for 5 seconds
downloading daily data for year 2001
data for year 2001 downloaded
sleep for 7 seconds
downloading daily data for year 2002
data for year 2002 downloaded
sleep for 8 seconds
downloading daily data for year 2003
data for year 2003 downloaded
sleep for 3 seconds
downloading daily data for year 2004
data for year 2004 downloaded
sleep for 9 seconds
downloading daily data for year 2005
data for year 2005 downloaded
sleep for 7 seconds
downloading daily data for year 2006
data for year 2006 downloaded
sleep for 9 seconds
downloading daily data for year 2007
data for year 2007 downloaded
sleep for 3 seconds
downloading daily data for year 2008
data for year 2008 downloaded
sleep for 5 seconds
downloading daily data for year 2009
data for year 2009 downloaded
sleep for 7 seconds
downloading daily data for year 2010
data for year 2010 downloaded
sleep for 3 seconds
down

In [3]:
weather_yyc_df['city'] = 'yyc'
weather_yeg_df['city'] = 'yeg'
weather_ymm_df['city'] = 'ymm'


In [4]:
print('yyc')
print(weather_yyc_df.describe())

print('yeg')
print(weather_yeg_df.describe())

print('ymm')
print(weather_ymm_df.describe())

yyc
              Year        Month          Day  Max Temp (°C)  Min Temp (°C)  \
count  7671.000000  7671.000000  7671.000000    3095.000000    3095.000000   
mean   2010.000000     6.522487    15.730935      11.029176      -0.884152   
std       6.056341     3.449049     8.801214      11.759673      10.072392   
min    2000.000000     1.000000     1.000000     -28.400000     -32.400000   
25%    2005.000000     4.000000     8.000000       3.100000      -7.300000   
50%    2010.000000     7.000000    16.000000      12.100000      -0.100000   
75%    2015.000000    10.000000    23.000000      20.600000       7.650000   
max    2020.000000    12.000000    31.000000      36.500000      17.500000   

       Mean Temp (°C)  Heat Deg Days (°C)  Cool Deg Days (°C)  \
count     3095.000000         3095.000000         3095.000000   
mean         5.083393           13.107593            0.190985   
std         10.721472           10.460052            0.731475   
min        -29.800000            

In [5]:
print('yyc')
print(weather_yyc_df.head())

print('yeg')
print(weather_yeg_df.head())

print('ymm')
print(weather_ymm_df.head())

yyc
   Date/Time  Year  Month  Day  Max Temp (°C)  Min Temp (°C)  Mean Temp (°C)  \
0 2000-01-01  2000      1    1            NaN            NaN             NaN   
1 2000-01-02  2000      1    2            NaN            NaN             NaN   
2 2000-01-03  2000      1    3            NaN            NaN             NaN   
3 2000-01-04  2000      1    4            NaN            NaN             NaN   
4 2000-01-05  2000      1    5            NaN            NaN             NaN   

   Heat Deg Days (°C)  Cool Deg Days (°C)  Total Rain (mm)  Total Snow (cm)  \
0                 NaN                 NaN              0.0              0.0   
1                 NaN                 NaN              0.0              0.0   
2                 NaN                 NaN              0.0              0.0   
3                 NaN                 NaN              0.0              0.0   
4                 NaN                 NaN              0.0              0.0   

   Total Precip (mm)  Snow on Grnd (cm) 

In [19]:
weather_yeg_df.head()

,Date/Time,Year,Month,Day,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Heat Deg Days (°C),Cool Deg Days (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Dir of Max Gust (10s deg),city,datetime
0,2000-01-01,2000,1,1,-10.2,-18.6,-14.4,32.4,0.0,0.0,2.4,2.4,0.0,0.0,yeg,2000-01-01
1,2000-01-02,2000,1,2,-17.1,-20.7,-18.9,36.9,0.0,0.0,0.0,0.0,1.0,0.0,yeg,2000-01-02
2,2000-01-03,2000,1,3,-15.5,-22.2,-18.9,36.9,0.0,0.0,0.0,0.0,1.0,0.0,yeg,2000-01-03
3,2000-01-04,2000,1,4,-12.8,-18.4,-15.6,33.6,0.0,0.0,0.0,0.0,1.0,0.0,yeg,2000-01-04
4,2000-01-05,2000,1,5,-5.1,-14.7,-9.9,27.9,0.0,0.0,1.0,1.0,2.0,0.0,yeg,2000-01-05


In [6]:
#join tables together

#dfs = [weather_yyc_df, weather_yeg_df, weather_ymm_df]
#weather_df = reduce(lambda left,right: pd.merge(left,right,on='Date/Time'), dfs)


In [7]:
print(weather_df.columns)
print(weather_df.head())



NameError: name 'weather_df' is not defined

In [10]:
weather_df = pd.concat([weather_yyc_df, weather_yeg_df, weather_ymm_df]).groupby(level=0).mean()

In [11]:
weather_df.columns

Index(['Year', 'Month', 'Day', 'Max Temp (°C)', 'Min Temp (°C)',
       'Mean Temp (°C)', 'Heat Deg Days (°C)', 'Cool Deg Days (°C)',
       'Total Rain (mm)', 'Total Snow (cm)', 'Total Precip (mm)',
       'Snow on Grnd (cm)', 'Dir of Max Gust (10s deg)'],
      dtype='object')

In [27]:
weather_df['datetime'] = weather_df['Year'].astype(str) + '-' + weather_df['Month'].astype(str) + '-' + weather_df['Day'].astype(str)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
#weather_df['datetime'] = weather_df['datetime'] datetime.strptime(weather_df['datetime'], '%Y-%m-%d')

In [31]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7671 entries, 0 to 7670
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Unnamed: 0                 7671 non-null   int64         
 1   Year                       7671 non-null   int64         
 2   Month                      7671 non-null   int64         
 3   Day                        7671 non-null   int64         
 4   Max Temp (°C)              7671 non-null   float64       
 5   Min Temp (°C)              7671 non-null   float64       
 6   Mean Temp (°C)             7671 non-null   float64       
 7   Heat Deg Days (°C)         7671 non-null   float64       
 8   Cool Deg Days (°C)         7671 non-null   float64       
 9   Total Rain (mm)            7671 non-null   float64       
 10  Total Snow (cm)            7671 non-null   float64       
 11  Total Precip (mm)          7671 non-null   float64       
 12  Snow o

In [22]:
weather_df.head()

,Unnamed: 0,Year,Month,Day,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Heat Deg Days (°C),Cool Deg Days (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Dir of Max Gust (10s deg),datetime
0,0,2000,1,1,-10.2,-18.6,-14.4,32.4,0.0,0.0,1.2,0.800000,0.0,0.0,2000-1-1
1,1,2000,1,2,-17.1,-20.7,-18.9,36.9,0.0,0.0,0.0,0.000000,0.5,0.0,2000-1-2
2,2,2000,1,3,-15.5,-22.2,-18.9,36.9,0.0,0.0,0.0,0.000000,0.5,0.0,2000-1-3
3,3,2000,1,4,-12.8,-18.4,-15.6,33.6,0.0,0.0,0.0,0.000000,0.5,0.0,2000-1-4
4,4,2000,1,5,-5.1,-14.7,-9.9,27.9,0.0,0.0,0.5,0.333333,1.0,0.0,2000-1-5


In [ ]:
weather_df.reset_index()
weather_df.describe

In [14]:
# write to csv to read in
weather_df.to_csv('weather_df.csv')

Run function "aeso_download_range" to scrape electricity daily average pool prices for time range

In [ ]:
#electricity params
tablename = 'DailyAveragePoolPrice'
startdate = '2000-01-01'
enddate = '2020-12-31'
dateformat = '%Y-%m-%d'
contenttype = 'html'
#get table of downloaded data, sort by date
util_prices = aeso_download_range(tablename, contenttype, startdate, enddate, dateformat).sort_values(by='$/MWh_Date').reset_index()

In [ ]:
# write to csv to read in
util_prices.to_csv('util_prices.csv')


In [15]:
# run this to read in csvs instead of needing to re download all the data
util_prices = pd.read_csv('util_prices.csv')
weather_df = pd.read_csv('weather_df.csv')




Join weather data and electricity price data together for analysis

In [35]:
# convert $/MWh_Date to date time
util_prices['$/MWh_Date'] = pd.to_datetime(util_prices['$/MWh_Date'])


In [36]:
unseen_weather_df = weather_df.iloc[6515:]
unseen_util_prices_df = util_prices.iloc[6515:]
train_weather_df = weather_df.iloc[:6514]
train_util_prices_df = util_prices.iloc[:6514]

In [37]:
#join tables together
train_df = train_util_prices_df.merge(train_weather_df, how = 'left', left_on = '$/MWh_Date', right_on = 'datetime')
unseen_df = unseen_util_prices_df.merge(unseen_weather_df, how = 'left', left_on = '$/MWh_Date', right_on = 'datetime')

Pick Daily_Average as the price to analyze (predictor label), and convert to kwh for standard consumer units Define columns used for prediction label, features and timestamp

In [ ]:
train_df['Avg_Price_kwh'] = train_df['Daily_Average'] / 1000
train_df = train_df.rename(columns = {'$/MWh_Date':'Date'})

unseen_df['Avg_Price_kwh'] = unseen_df['Daily_Average'] / 1000
unseen_df = unseen_df.rename(columns = {'$/MWh_Date':'Date'})

In [ ]:
print(train_df.info())
print(unseen_df.info())

print(train_df.head())
print(unseen_df.head())


In [ ]:


train_df = train_df[['Date','Month', 'Day', 'Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)',
            'Heat Deg Days (°C)', 'Cool Deg Days (°C)', 'Total Rain (mm)',
            'Total Snow (cm)', 'Total Precip (mm)', 'Snow on Grnd (cm)',
            'Dir of Max Gust (10s deg)','Avg_Price_kwh']]
unseen_df = unseen_df[['Date','Month', 'Day', 'Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)',
            'Heat Deg Days (°C)', 'Cool Deg Days (°C)', 'Total Rain (mm)',
            'Total Snow (cm)', 'Total Precip (mm)', 'Snow on Grnd (cm)',
            'Dir of Max Gust (10s deg)','Avg_Price_kwh']]


In [ ]:
dataset = train_df
#check the shape of data
dataset.shape

In [ ]:

data = dataset
data_unseen = unseen_df


print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'Avg_Price_kwh', session_id=1)

In [ ]:
best = compare_models(exclude = ['ransac'])

In [ ]:
models()

In [ ]:
model_selected = "gbr"

In [ ]:
gbr = create_model('gbr')
 

In [ ]:
tuned_gbr = tune_model(gbr)


In [ ]:
plot_model(tuned_gbr)

In [ ]:
plot_model(tuned_gbr, plot = 'feature')

In [ ]:
plot_model(tuned_gbr, plot = 'feature')


In [ ]:
predict_model(gbr)

In [ ]:

final_gbr = finalize_model(gbr)

In [ ]:
unseen_predictions = predict_model(final_gbr, data=data_unseen)
unseen_predictions.head()

In [ ]:
%matplotlib inline
unseen_predictions_label = unseen_predictions[['Avg_Price_kwh','Label']]
unseen_predictions_label.plot(ylim=[0,0.15], linewidth=1, alpha=0.7)

In [ ]:
unseen_predictions_label['diff'] = unseen_predictions_label['Avg_Price_kwh'] - unseen_predictions_label['Label']
unseen_predictions_label.plot(ylim=[-0.04,0.10], linewidth=1, alpha=0.7)



In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions.Avg_Price_kwh, unseen_predictions.Label, 'R2')

In [ ]:
save_model(final_rf,'Final gbr Model 28Mar2021')